### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [7]:
num_iterations = 1
empty_list = []
for i in range(1200, 1250):
    if i == 724:
        continue

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = EB_EC
class_probability = 0.29


######### Lightcurve features #########

amplitude = 0.08325918315797937
cusum = 0.30769814295715386
eta = 2.63705311847211
hl_amp_ratio = 1.241449874238752
kurtosis = -0.07281117369290424
n_points = 1096
period = 0.2763658689110861
period_SNR = 163.3187749150427
period_log10FAP = -90.28689426496281
period_uncertainty = 2.6737149432493057e-05
phase_cusum = 0.3383067121952038
phase_eta = 1.590187182328599
phi21 = -3.289717434876059
phi31 = -3.31237858541598
quartile31 = 0.13505914017149756
r21 = 0.1039506570409264
r31 = 0.05847916408736171
shapiro_w = 0.9691506624221802
skewness = 0.5717655377420758
slope_per10 = -0.028013573159012064
slope_per90 = 0.031094070349048698
stetson_k = 0.8658268309694418
weighted_mean = 11.690566708070191
weighted_std = 0.07815928972569355
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3041327639829161
cusum = 0.15295593899909962
eta = 2.420768856038149
hl_amp_ratio = 0.3703382631016014
kurtosis = -0.8631312720631912
n_points = 1105
period = 0.6046156222990218
period_SNR = 197.65468259441315
period_log10FAP = -164.40145980092328
period_uncertainty = 0.00015479151611064168
phase_cusum = 0.33340636270474844
phase_eta = 0.15151597087875174
phi21 = 0.6344439335050786
phi31 = 1.7042701978975303
quartile31 = 0.41685641603130463
r21 = 0.43909368581462593
r31 = 0.3086093880284898
shapiro_w = 0.9134929180145264
skewness = -0.5929913607922742
slope_per10 = -0.08416116912788128
slope_per90 = 0.08770043045573041
stetson_k = 0.7298482799738788
weighted_mean = 14.615802984938021
weighted_std = 0.2505997602296533
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4325470967888139
cusum = 0.16645882874477463
eta = 1.6029388954932602
hl_amp_ratio = 0.3364106311821619
kurtosis = -0.38458965409334045
n_points = 1107
period = 0.48951248967771577
period_SNR = 208.6323679956673
period_log10FAP = -160.78534380491143
period_uncertainty = 8.321080153286808e-05
phase_cusum = 0.27629316996641895
phase_eta = 0.2726204024095613
phi21 = -2.4434353614087874
phi31 = -1.5922505068809574
quartile31 = 0.5799935455568459
r21 = 0.42176916432673167
r31 = 0.2273815015247933
shapiro_w = 0.9038582444190979
skewness = -0.8245604833782658
slope_per10 = -0.023777372489608702
slope_per90 = 0.022739802187153393
stetson_k = 0.6888255054133796
weighted_mean = 14.458914136242761
weighted_std = 0.3762272066541654
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3498842939670841
cusum = 0.18248496524961957
eta = 2.409969664405134
hl_amp_ratio = 0.4412453723179078
kurtosis = -0.8874109288753882
n_points = 1241
period = 0.6975756410236355
period_SNR = 212.08426777297998
period_log10FAP = -181.07607485369223
period_uncertainty = 0.0002044078045284059
phase_cusum = 0.36339423187185593
phase_eta = 0.06063946948728276
phi21 = -2.260809134481394
phi31 = -1.125694736173188
quartile31 = 0.47852003303775703
r21 = 0.5169367581673512
r31 = 0.3213079626933928
shapiro_w = 0.9281964898109436
skewness = -0.5456675124102852
slope_per10 = -0.07446781187720694
slope_per90 = 0.0761694965743361
stetson_k = 0.7282494581112969
weighted_mean = 14.673713007082851
weighted_std = 0.2913451890488742


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
